In [1]:
import pandas as pd
import numpy as np
import datetime
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder, TargetEncoder
import lightgbm as lgb
# import optuna.integration.lightgbm as lgb

import category_encoders as ce
import math

# pandasの行を省略しない
pd.set_option('display.max_columns', None)

In [2]:
train_df = pd.read_csv('C:/python/signate/data/train.csv', index_col=0)
test_df = pd.read_csv('C:/python/signate/data/test.csv', index_col=0)
sample_df = pd.read_csv('C:/python/signate/data/sample_submission.csv', header=None, index_col=0)
train_df.head()

,created_at,tree_dbh,curb_loc,health,steward,guards,sidewalk,user_type,problems,spc_common,spc_latin,nta,nta_name,borocode,boro_ct,boroname,zip_city,cb_num,st_senate,st_assem,cncldist
0,2015-06-29,14,OnCurb,1,NaN,NaN,Damage,Volunteer,NaN,English oak,Quercus robur,QN45,Douglas Manor-Douglaston-Little Neck,4,4152901,Queens,Little Neck,411,11,25,23
1,2016-09-21,5,OnCurb,1,3or4,Helpful,NoDamage,Volunteer,NaN,crimson king maple,Acer platanoides 'Crimson King',BX05,Bedford Park-Fordham North,2,2039901,Bronx,Bronx,207,33,78,15
2,2015-09-13,26,OnCurb,2,NaN,NaN,NoDamage,Volunteer,StonesBranchLights,English oak,Quercus robur,SI01,Annadale-Huguenot-Prince's Bay-Eltingville,5,5017011,Staten Island,Staten Island,503,24,62,51
3,2016-05-09,15,OnCurb,0,NaN,NaN,Damage,NYC Parks Staff,NaN,honeylocust,Gleditsia triacanthos var. inermis,SI11,Charleston-Richmond Valley-Tottenville,5,5024401,Staten Island,Staten Island,503,24,62,51
4,2016-06-24,23,OnCurb,1,NaN,NaN,NoDamage,Volunteer,Stones,London planetree,Platanus x acerifolia,MN03,Central Harlem North-Polo Grounds,1,1022102,Manhattan,New York,110,30,70,9


データはほとんどカテゴリカル
#### 思いついた精度向上案
- commonを個別名称として、latinの上を属名として利用する
- 地区が多すぎるので整理する
- 季節性を導入
- 郵便番号は連続性があるからカテゴリカルにしないほうがよいのでは
- 高い確率で状態が悪いと推測された木の近くの木はフラグ立てる。つまり2段階モデル
- 同じ人が記録した場合、1日のうちの0,1,2の割合は無意識にバイアスがかかって同じくらいにしてしまうのでは？
- 曜日の導入

In [3]:
def cleansing(df):
    df['created_at'] = pd.to_datetime(df['created_at'])
    # df['cos_day'] = df['created_at'].dt.dayofyear
    # df['cos_day'] = df['cos_day'].apply(lambda x: np.cos(math.radians(90 - (x/365)*365)))
    # df['sin_day'] = df['created_at'].dt.dayofyear
    # df['sin_day'] = df['sin_day'].apply(lambda x: np.sin(math.radians(90 - (x/365)*365)))  
    # df['year'] = df['created_at'].dt.year
    # df['month'] = df['created_at'].dt.month
    # df['weekday'] = df['created_at'].dt.weekday
    # df['day'] = df.created_at.dt.day
    # df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    # df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df.drop('created_at', axis=1, inplace=True)
    df['steward'].fillna('0', inplace=True)
    df['guards'].fillna('0', inplace=True)
    df['problems'].fillna('NoProblem', inplace=True)
    df['spc_genus'] = df['spc_latin'].str.split(' ').str[0]
    df.drop('spc_latin', axis=1, inplace=True)

    # df['curb_loc'] = df['curb_loc'].map({'OnCurb':3, 'OffsetFromCurb':1})
    # df['sidewalk'] = df['sidewalk'].map({'NoDamage':1, 'Damage':3})
    # df['guards'] = df['guards'].map({'Helpful':1, 'Harmful':3, '0':2, 'unsure':2})
    # df['status_point'] = df['curb_loc'] * df['sidewalk'] * df['guards']
    return df

train_clean = cleansing(train_df)
test_clean = cleansing(test_df)
train_clean.head()

,tree_dbh,curb_loc,health,steward,guards,sidewalk,user_type,problems,spc_common,nta,nta_name,borocode,boro_ct,boroname,zip_city,cb_num,st_senate,st_assem,cncldist,spc_genus
0,14,OnCurb,1,0,0,Damage,Volunteer,NoProblem,English oak,QN45,Douglas Manor-Douglaston-Little Neck,4,4152901,Queens,Little Neck,411,11,25,23,Quercus
1,5,OnCurb,1,3or4,Helpful,NoDamage,Volunteer,NoProblem,crimson king maple,BX05,Bedford Park-Fordham North,2,2039901,Bronx,Bronx,207,33,78,15,Acer
2,26,OnCurb,2,0,0,NoDamage,Volunteer,StonesBranchLights,English oak,SI01,Annadale-Huguenot-Prince's Bay-Eltingville,5,5017011,Staten Island,Staten Island,503,24,62,51,Quercus
3,15,OnCurb,0,0,0,Damage,NYC Parks Staff,NoProblem,honeylocust,SI11,Charleston-Richmond Valley-Tottenville,5,5024401,Staten Island,Staten Island,503,24,62,51,Gleditsia
4,23,OnCurb,1,0,0,NoDamage,Volunteer,Stones,London planetree,MN03,Central Harlem North-Polo Grounds,1,1022102,Manhattan,New York,110,30,70,9,Platanus


In [4]:

# 種目ごとの平均直径とその差を特徴量にする
test_clean['health'] = 0
all_df = pd.concat([train_clean, test_clean], axis=0)   
tmp_df = all_df[['tree_dbh', 'spc_common', ]]
dbh_mean_common = (pd.DataFrame(tmp_df.groupby('spc_common').mean()['tree_dbh']))
dbh_mean_common.rename(columns={'tree_dbh': 'dbh_mean_common'}, inplace=True)
all_df = pd.merge(all_df, dbh_mean_common, on='spc_common', how='left')
# all_df['dbh_diff_common'] = all_df['tree_dbh'] - all_df['dbh_mean_common']

tmp_df = all_df[['tree_dbh', 'cb_num']]
dbh_mean_cb = (pd.DataFrame(tmp_df.groupby('cb_num').mean()['tree_dbh']))
dbh_mean_cb.rename(columns={'tree_dbh': 'dbh_mean_cb'}, inplace=True)
all_df = pd.merge(all_df, dbh_mean_cb, on='cb_num', how='left')


train_clean = all_df.iloc[:len(train_clean)]
test_clean = all_df.iloc[len(train_clean):]
test_clean.drop('health', axis=1, inplace=True)
test_clean.head()


C:\Users\koba_\AppData\Local\Temp\ipykernel_34396\3612706299.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_clean.drop('health', axis=1, inplace=True)


,tree_dbh,curb_loc,steward,guards,sidewalk,user_type,problems,spc_common,nta,nta_name,borocode,boro_ct,boroname,zip_city,cb_num,st_senate,st_assem,cncldist,spc_genus,dbh_mean_common,dbh_mean_cb
19984,15,OnCurb,0,0,Damage,NYC Parks Staff,NoProblem,Callery pear,BK17,Sheepshead Bay-Gerritsen Beach-Manhattan Beach,3,3062000,Brooklyn,Brooklyn,315,22,45,48,Pyrus,9.993088,11.813235
19985,5,OnCurb,1or2,0,NoDamage,Volunteer,NoProblem,cherry,BX62,Woodlawn-Wakefield,2,2045101,Bronx,Bronx,212,34,81,11,Prunus,6.937519,13.062044
19986,4,OnCurb,0,Unsure,NoDamage,Volunteer,NoProblem,littleleaf linden,QN60,Kew Gardens,4,4013800,Queens,Kew Gardens,409,14,27,29,Tilia,9.373364,11.174312
19987,7,OnCurb,0,0,NoDamage,TreesCount Staff,NoProblem,dawn redwood,BK09,Brooklyn Heights-Cobble Hill,3,3000301,Brooklyn,Brooklyn,302,26,52,33,Metasequoia,9.541667,9.894670
19988,6,OnCurb,1or2,Helpful,NoDamage,TreesCount Staff,Stones,purple-leaf plum,BK75,Bedford,3,3025100,Brooklyn,Brooklyn,303,25,56,36,Prunus,9.447942,12.236810


In [5]:
'''
# trainとtestに含まれるproblemは全く同じ
# problemのonehotカラムを用意
problem_list = ['BranchLights', 'BranchOther', 'MetalGrates', 'RootOther', 'Stones', 'Sneakers', 'TrunkLights',
                 'TrunkOther', 'WiresRope', 'NoProblem']	

for problem in problem_list:
    train_clean.loc[:, problem] = 0
    test_clean.loc[:, problem] = 0

train_clean['problem_count'] = 0
test_clean['problem_count'] = 0

# problemlistにあったらonehotする
for i in train_clean.index:
    p_count = 0
    for problem in problem_list:
        if(problem in train_clean.loc[i, 'problems']):
            train_clean.loc[i, problem] = 1
            if(problem != 'NoProblem'):
                p_count+=1
    train_clean.loc[i, 'problem_count'] = p_count

for i in test_clean.index:
    p_count = 0
    for problem in problem_list:
        if(problem in test_clean.loc[i, 'problems']):
            test_clean.loc[i, problem] = 1
            if(problem != 'NoProblem'):
                p_count+=1
    test_clean.loc[i, 'problem_count'] = p_count

test_clean.head()
'''

"\n# trainとtestに含まれるproblemは全く同じ\n# problemのonehotカラムを用意\nproblem_list = ['BranchLights', 'BranchOther', 'MetalGrates', 'RootOther', 'Stones', 'Sneakers', 'TrunkLights',\n                 'TrunkOther', 'WiresRope', 'NoProblem']\t\n\nfor problem in problem_list:\n    train_clean.loc[:, problem] = 0\n    test_clean.loc[:, problem] = 0\n\ntrain_clean['problem_count'] = 0\ntest_clean['problem_count'] = 0\n\n# problemlistにあったらonehotする\nfor i in train_clean.index:\n    p_count = 0\n    for problem in problem_list:\n        if(problem in train_clean.loc[i, 'problems']):\n            train_clean.loc[i, problem] = 1\n            if(problem != 'NoProblem'):\n                p_count+=1\n    train_clean.loc[i, 'problem_count'] = p_count\n\nfor i in test_clean.index:\n    p_count = 0\n    for problem in problem_list:\n        if(problem in test_clean.loc[i, 'problems']):\n            test_clean.loc[i, problem] = 1\n            if(problem != 'NoProblem'):\n                p_count+=1\n    test_clean.

## やりたい実験
- あらゆるエンコーディング方法の比較 ok
- 地区ごとの予測モデルの実装 
- 365日で一周するsin, cos 意味無し


In [6]:
train_clean.columns

Index(['tree_dbh', 'curb_loc', 'health', 'steward', 'guards', 'sidewalk',
       'user_type', 'problems', 'spc_common', 'nta', 'nta_name', 'borocode',
       'boro_ct', 'boroname', 'zip_city', 'cb_num', 'st_senate', 'st_assem',
       'cncldist', 'spc_genus', 'dbh_mean_common', 'dbh_mean_cb'],
      dtype='object')

In [7]:
# エンコーディングパート

# すべてのカテゴリカル変数
all_categorical_cols = ['curb_loc', 'steward', 'guards', 'sidewalk',
       'user_type', 'problems', 'spc_common', 'nta', 'nta_name', 'borocode',
       'boro_ct', 'boroname', 'zip_city', 'cb_num', 'st_senate', 'st_assem',
       'cncldist', 'year', 'month',  'spc_genus', ]

# 

# 落とすカラム
drop_cols = ['nta_name', 'boroname']#'',
# カウントエンコーディング
ce_columns = ['curb_loc', 'steward', 'guards', 'sidewalk',
       'user_type', 'problems', 'spc_common', 'nta', 
              'borocode', 'boro_ct',  'zip_city', 'cb_num', 'st_senate',
       'st_assem', 'cncldist', 'spc_genus']
# 'nta',
#  
# 

# ターゲットエンコーディング
te_columns = []
# ラベルエンコーディング
le_columns = []
encoding_cals = list(drop_cols + ce_columns + te_columns + drop_cols)
if(encoding_cals != all_categorical_cols):
    print('error! categorical is not same')
    print('足りないカラム:', set(all_categorical_cols)-set(encoding_cals))

error! categorical is not same
足りないカラム: {'month', 'year'}


In [8]:
# ターゲットエンコーディングパート
if(len(te_columns) > 0):
    # onehot正解ラベルの作成
    for i in range(3):
        train_clean['onehot' + str(i)] = 0
        train_clean['onehot' + str(i)] = train_clean['onehot' + str(i)].mask(train_clean['health'] == i, 1)

    # ターゲットエンコーディングのカラムを3つずつに分ける
    te_columns_list = []

    for te_column in te_columns:
        tmp_list = []
        for i in range(3):
            train_clean[te_column + '-te' + str(i)] = train_clean[te_column]
            test_clean[te_column + '-te' + str(i)] = test_clean[te_column]
            tmp_list.append(te_column + '-te' + str(i))
        te_columns_list.append(tmp_list)
        train_clean.drop(te_column, axis=1, inplace=True)
        test_clean.drop(te_column, axis=1, inplace=True)

    te_columns_list = np.array(te_columns_list)

    # ターゲットエンコーディング
    for i in range(3):
        te_target = te_columns_list[:, i]
        target_enc = TargetEncoder(target_type='binary', random_state=42)
        
        train_clean[te_target] = target_enc.fit_transform(train_clean[te_target], train_clean['onehot' + str(i)])
        # テストデータにtransform
        test_clean[te_target]  = target_enc.transform(test_clean[te_target])

    # onehot正解ラベルを削除
    for i in range(3):
        dropcol = 'onehot' + str(i)
        train_clean.drop(dropcol, axis=1, inplace=True)


# 目的変数を指定
y = train_clean['health']

# 使わないcolumnを削除
train_clean_drop = train_clean.drop(['health'], axis=1)
train_clean_drop = train_clean_drop.drop(drop_cols, axis=1)
test_clean_drop = test_clean.drop(drop_cols,axis=1)

# ラベルエンコーディングパート
le = LabelEncoder()
for column in le_columns:
    train_clean_drop[column] = le.fit_transform(train_clean_drop[column])
    test_clean_drop[column] = le.transform(test_clean_drop[column])

# カテゴリカル変数を指定
for col in le_columns:
    train_clean_drop[col] = train_clean_drop[col].astype('category')
    test_clean_drop[col] = test_clean_drop[col].astype('category')  

# カウントエンコーディングパート
# mergeしてfit, そのあとtransform
all_df = pd.concat([train_clean_drop, test_clean_drop], axis=0)

# count encoding
cencoder = ce.CountEncoder(cols=ce_columns)
cencoder.fit(all_df)

# count encodingを適用
train_clean_drop = cencoder.transform(train_clean_drop)
test_clean_drop = cencoder.transform(test_clean_drop)

In [9]:
train_clean_drop.head()

,tree_dbh,curb_loc,steward,guards,sidewalk,user_type,problems,spc_common,nta,borocode,boro_ct,zip_city,cb_num,st_senate,st_assem,cncldist,spc_genus,dbh_mean_common,dbh_mean_cb
0,14,37270,29409,29510,18509,22298,24288,2579,413,13234,129,413,1654,3780,1145,1694,7853,12.346646,11.311971
1,5,37270,2068,7366,21177,22298,24288,254,59,4479,24,4479,166,564,162,380,3613,10.157480,16.289157
2,26,37270,29409,29510,21177,22298,1529,2579,616,7292,189,7292,3436,5720,2868,3155,7853,12.346646,9.683062
3,15,37270,29409,29510,18509,6031,24288,2104,678,7292,121,7292,3436,5720,2868,3155,2104,11.484316,9.683062
4,23,37270,29409,29510,21177,22298,4455,4339,144,4522,39,4473,289,948,418,542,4339,19.432819,11.678201


In [10]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
valid_scores = []
models = []
# train_clean_dropと同じindexを持つデータフレームを作成
lgb_pred_proba = train_df[['health', 'tree_dbh', 'nta']].copy()
lgb_pred_proba.rename(columns= {'health':'lgb-0', 'tree_dbh':'lgb-1', 'nta':'lgb-2'}, inplace=True)

for fold, (train_idx, valid_idx) in enumerate(kf.split(train_clean_drop, y)):
    print(f'fold{fold} start')
    X_train, y_train = train_clean_drop.iloc[train_idx], y.iloc[train_idx]
    X_valid, y_valid = train_clean_drop.iloc[valid_idx], y.iloc[valid_idx]

    lgb_train = lgb.Dataset(X_train, y_train, weight=compute_sample_weight(class_weight='balanced', y=y_train).astype('float32'))
    lgb_eval = lgb.Dataset(X_valid, y_valid, weight=np.ones(len(X_valid)).astype('float32'))

    '''
    params = {'objective': 'multiclass',
 'metric': 'multi_logloss',
 'num_class': 3,
 'seed': 0,
 'feature_pre_filter': False,
 'lambda_l1': 0.0015923361719036968,
 'lambda_l2': 0.004786876640032096,
 'num_leaves': 252,
 'feature_fraction': 0.8,
 'bagging_fraction': 0.9931160872840541,
 'bagging_freq': 7,
 'min_child_samples': 5,
 'num_iterations': 1000}
 '''
    verbose_eval = -1 # この数字を1にすると学習時のスコア推移がコマンドライン表示される
    params = {'objective': 'multiclass',
 'metric': 'multi_logloss',
 'num_class': 3,
 'seed': 0,}
    
    model = lgb.train(params,
                      lgb_train,
                      valid_sets=[lgb_train, lgb_eval],
                      num_boost_round=1000,
                       callbacks=[lgb.early_stopping(stopping_rounds=50, 
                                verbose=True), # early_stopping用コールバック関数
                           lgb.log_evaluation(verbose_eval)], # コマンドライン出力用コールバック関数))
                        # feval=f1,
                        # categorical_feature=le_columns,
                    )

    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    lgb_pred_proba.iloc[valid_idx] = y_pred
    y_pred_max = np.argmax(y_pred, axis=1)
    score = f1_score(y_valid, y_pred_max, average='macro')
    print(f'fold{fold} f1_score: {score:.4f}')
    valid_scores.append(score)
    models.append(model)
print(f'CV: {np.mean(valid_scores):.4f}')
lgb_pred_proba.to_csv('C:/python/signate/data/lgb_pred_proba.csv')


fold0 start


LightGBMError: Length of weights is not same with #data

In [ ]:
# 全データを利用して学習
'''
lgb_train = lgb.Dataset(train_clean_drop, y, weight=compute_sample_weight(class_weight='balanced', y=y).astype('float32'))
verbose_eval = -1 # この数字を1にすると学習時のスコア推移がコマンドライン表示される
params = {'objective': 'multiclass',
'metric': 'multi_logloss',
'num_class': 3,
'seed': 0
}

model = lgb.train(params,
                    lgb_train,
                    num_boost_round=520,
                )
'''

"\nlgb_train = lgb.Dataset(train_clean_drop, y, weight=compute_sample_weight(class_weight='balanced', y=y).astype('float32'))\nverbose_eval = -1 # この数字を1にすると学習時のスコア推移がコマンドライン表示される\nparams = {'objective': 'multiclass',\n'metric': 'multi_logloss',\n'num_class': 3,\n'seed': 0\n}\n\nmodel = lgb.train(params,\n                    lgb_train,\n                    num_boost_round=520,\n                )\n"

In [ ]:
lgb_pred_proba_test = test_df[['borocode', 'tree_dbh', 'nta']].copy()
lgb_pred_proba_test.rename(columns= {'borocode':'lgb-0', 'tree_dbh':'lgb-1', 'nta':'lgb-2'}, inplace=True)

y_pred = model.predict(test_clean_drop)
lgb_pred_proba_test.iloc[:] = y_pred[:]
lgb_pred_proba_test.to_csv('C:/python/signate/data/lgb_pred_proba_test.csv')
y_pred_max = np.argmax(y_pred, axis=1) # 最尤と判断したクラスの値にする
sample_df[1] = y_pred_max
# sample_df.to_csv('C:/python/signate/data/base3(cv0.3586).csv', header=None)

In [ ]:
import pandas as pd

importance = model.feature_importance()
feature_names = model.feature_name()

df_importance = pd.DataFrame({'Feature': feature_names, 'Importance': importance})
# importtane順に並び替え
df_importance = df_importance.sort_values('Importance', ascending=False)
df_importance


,Feature,Importance
0,tree_dbh,6420
10,boro_ct,5888
8,nta,3952
7,spc_common,3606
17,dbh_mean_common,3276
6,problems,3234
16,spc_genus,2577
14,st_assem,2499
18,dbh_mean_cb,2136
12,cb_num,1979
